In [86]:
import torch 
from torch import nn
from torch import optim
import keras
import numpy as np
from model import TextRNN
from cnews_loader import read_category, read_vocab, process_file
import torch.utils.data as Data

In [87]:
train_file = 'cnews.train.txt'
test_file = 'cnews.test.txt'
val_file = 'cnews.val.txt'
vocab_file = 'cnews.vocab.txt'

def train():
    #use TextRNN model
    model = TextRNN().cuda()
    #define loss func
    Loss = nn.MultiLabelSoftMarginLoss()
    optimizer = optim.Adam(model.parameters(),lr = 0.001)
    best_val_acc = 0
    
    for epoch in range(10):
        print('epoch=', epoch)
        for step, (x_batch, y_batch) in enumerate(train_loader):
            x = x_batch.cuda()
            y = y_batch.cuda()
            out = model(x)
            loss = Loss(out,y)
            print('loss = ', loss)
            
            #bachprop
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            #calculate accuracy
            accuracy = np.mean((torch.argmax(out, 1) == torch.argmax(y, 1)).cpu().numpy())
            print(accuracy)
            
            #validation
            if(epoch+1)%5 == 0:
                for step, (x_batch, y_batch) in enumerate(val_loader):
                    x = x_batch.cuda()
                    y = y_batch.cuda()
                    out = model(x)
                    
                    #calculate accuracy
                    accuracy = np.mean((torch.argmax(out, 1) == torch.argmax(y, 1)).cpu().numpy())
                    if accuracy > best_val_acc:
                        torch.save(model.state_dict(), 'model_params.pkl')
                        best_val_acc = accuracy
                        print(accuracy)

In [88]:
# 获取文本的类别及其对应id的字典
categories, cat_to_id = read_category()
#print(categories)
# 获取训练文本中所有出现过的字及其所对应的id
words, word_to_id = read_vocab('cnews.vocab.txt')
#print(words)
#print(word_to_id)
#获取字数
vocab_size = len(words)

In [89]:
###### 数据加载及分批
# 获取训练数据每个字的id和对应标签的one-hot形式
x_train, y_train = process_file('cnews.train.txt', word_to_id, cat_to_id, 600)
#print('x_train=', x_train)
x_val, y_val = process_file('cnews.val.txt', word_to_id, cat_to_id, 600)

x_train, y_train = torch.LongTensor(x_train), torch.Tensor(y_train)
x_val, y_val = torch.LongTensor(x_val), torch.Tensor(y_val)

train_dataset = Data.TensorDataset(x_train, y_train)
train_loader = Data.DataLoader(dataset = train_dataset, batch_size = 100, shuffle = True, num_workers = 2)

val_dataset = Data.TensorDataset(x_val, y_val)
val_loader = Data.DataLoader(dataset = val_dataset, batch_size = 100, shuffle = True, num_workers = 2)

In [90]:
#before loading the model, release the GPU first
torch.cuda.empty_cache()

#load the model
train()

epoch= 0


RuntimeError: size mismatch, m1: [100 x 128], m2: [256 x 128] at C:/w/b/windows/pytorch/aten/src\THC/generic/THCTensorMathBlas.cu:283